# Прогнозирование стоимости автомобиля по характеристикам


Образовательная платформа: SkillFactory

Специализация: Data Science

Группа: DST-37 и 38

Юнит 6. Проект 5: "Выбираем автомобиль правильно"


### Задача:

    Создать модель, которая будет предсказывать стоимость автомобиля по его характеристикам для того, чтобы выявлять выгодные предложения (когда желаемая цена продавца ниже предсказанной рыночной цены).

### Метрика:

    MAPE (Mean Percentage Absolute Error) - средняя абсолютная ошибка в процентах

### Нужно:

    Составить train датасет - спарсить данные, либо найти готовый
    Обучить модель

### Плюс:

    Посмотреть, что можно извлечь из признаков или как еще можно обработать признаки
    Сгенерировать новые признаки
    Подгрузить еще больше данных
    Попробовать подобрать параметры модели
    Попробовать разные алгоритмы и библиотеки ML
    Сделать Ансамбль моделей, Blending, Stacking

### Этапы работы:

    Парсинг с авто.ру - Вадим
    EDA, Feature Engineering - Артём, Вадим, Женя
    Сравнение одиночных моделей - Артём, Вадим
    Стекинг - Вадим

#### В данном ноутбуке мы проводим парсинг данных с сайта auto.ru

#### Также в этом проекте мы использовали:

Спарсенный датасет мы взяли у этой команды, потому что парсинг занимал очень много времени: https://www.kaggle.com/juliadeinego/data-car-sales

Ноутбук, в котором провели EDA: https://www.kaggle.com/artemskakun/sf-dst-car-price-prediction-datapreprocessing

Ноутбук, в котором провели ML: https://www.kaggle.com/artemskakun/sf-dst-car-price-prediction-ml

In [6]:
#!pip install -U selenium
#!pip install beautifulsoup4

     |████████████████████████████████| 904 kB 5.9 MB/s eta 0:00:01


In [7]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import pandas as pd
import re
import pdb
from selenium import webdriver

BASE_URL = 'https://auto.ru'
FROM_YEAR = 2000
MARKS_TO_PARSE = ['SKODA', 'AUDI', 'HONDA', 'VOLVO', 'BMW', 'NISSAN', 'INFINITI',
                  'MERCEDES', 'TOYOTA', 'LEXUS', 'VOLKSWAGEN', 'MITSUBISHI']

### Функции

In [ ]:
# функция записи выбранных данных
def write_csv(trend, parsing_unixtime, models, prices, years, mileages, bodyTypes, colors, engines,
              transmissions, drives, wheels, states, owners, ptss):

    lenlist = len(models)
    i = 0

    with open(f'{PROJECT_PATH}data\\{trend}.csv', 'a', encoding="utf-8", newline='') as f:
        writer = csv.writer(f)

        while i <= (lenlist-1):
            writer.writerow([trend, parsing_unixtime, models[i], prices[i], years[i], mileages[i], bodyTypes[i],
                             colors[i], engines[i], transmissions[i], drives[i], wheels[i], states[i], owners[i], ptss[i]])
            i = i+1

# функция очистки текста от лишних символов
def clear_text(elem):
    if elem:
        if elem == "NaN":
            return elem
        return elem.text\
            .replace("&nbsp;", "")\
            .replace("₽", "")\
            .replace("\xa0", "")\
            .replace("\u2009", "")\
            .replace("/", "|")
    else:
        return "NaN"
    return models, prices, years, kms, descs

# функция вычленения текста из линка
def get_info_from_link(el, cls):
    span = get_info_from_span(el, cls)
    if span:
        return span.find('a', class_='Link_color_black')
    return "NaN"

# функция вычленения текста из спана
def get_info_from_span(el, cls):
    if el:
        li = el.find('li', class_=cls)
        if li:
            spans = li.find_all('span', class_='CardInfoRow__cell')
            if len(spans) == 1:
                return spans[0]
            elif len(spans) > 1:
                return spans[1]
    return "NaN"

# функция проходит по странице брэнда, переходит на страницу объявления. получив данные, сохраняет в списках признаков
def get_page_data(html):
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')
    car_rows = soup.find_all('div', class_='ListingCars-module__listingItem')
    print(f'now found {len(car_rows)} on the page')
    models = []
    prices = []
    years = []
    mileages = []
    bodyTypes = []
    colors = []
    engines = []
    transmissions = []
    drives = []
    wheels = []
    states = []
    owners = []
    ptss = []

    for car in car_rows:
        info = {}
        price = 0
        try:
            model = car.find('a', class_='ListingItemTitle-module__link')
            model_full_name = model.text
            model_url = model['href']
            # print(model_full_name,model_url)
            browser.get(model_url)
            html = browser.page_source
            price = clear_text(
                car.find('div', class_='ListingItemPrice-module__content'))

            info = get_model_data(model_url, html)
        except:
            print('Ошибка при парсинге', model_full_name, model_url)
            continue

        # print(info)
        prices.append(price)
        models.append(info['model'])
        years.append(info['year'])
        mileages.append(info['mileage'])
        bodyTypes.append(info['bodyType'])
        colors.append(info['color'])
        engines.append(info['engine'])
        transmissions.append(info['transmission'])
        drives.append(info['drive'])
        wheels.append(info['wheel'])
        states.append(info['state'])
        owners.append(info['owners_count'])
        ptss.append(info['pts'])

    return models, prices, years, mileages, bodyTypes, colors, engines, \
        transmissions, drives, wheels, states, owners, ptss

# функция получает данные со страницы объявления
def get_model_data(url, html):
    soup = BeautifulSoup(html, 'html.parser')
    cardInfo = soup.find('ul', class_='CardInfo')

    model_links = soup.find_all('a', class_='CardBreadcrumbs__itemText')
    model = ''
    if len(model_links) >= 4:
        model = clear_text(model_links[2])
        model += '|' + clear_text(model_links[3])
    elif soup.find('h1', class_='CardHead-module__title'):
        model = soup.find('h1', class_='CardHead-module__title').text
    else:
        print('problem in', model, model_links)

    year = clear_text(get_info_from_link(cardInfo, 'CardInfoRow_year'))
    if year == "NaN":
        print('Error found on link:', url)

    mileage = clear_text(get_info_from_span(cardInfo, 'CardInfoRow_kmAge'))
    bodyType = clear_text(get_info_from_link(cardInfo, 'CardInfoRow_bodytype'))
    color = clear_text(get_info_from_link(cardInfo, 'CardInfoRow_color'))
    engine = get_info_from_span(cardInfo, 'CardInfoRow_engine')
    engine_txt = ''
    if engine.find('div'):
        engine_txt = clear_text(engine.find('div'))
    else:
        engine_txt = clear_text(engine)
    transmission = clear_text(get_info_from_span(
        cardInfo, 'CardInfoRow_transmission'))
    drive = clear_text(get_info_from_span(cardInfo, 'CardInfoRow_drive'))
    wheel = clear_text(get_info_from_span(cardInfo, 'CardInfoRow_wheel'))
    state = clear_text(get_info_from_span(cardInfo, 'CardInfoRow_state'))
    owners_count = clear_text(get_info_from_span(
        cardInfo, 'CardInfoRow_ownersCount'))
    pts = clear_text(get_info_from_span(cardInfo, 'CardInfoRow_pts'))
    time.sleep(0.1)

    return {'model': model, 'year': year, 'mileage': mileage, 'bodyType': bodyType, 'color': color, 'engine': engine_txt, 'transmission': transmission, 'drive': drive, 'wheel': wheel, 'state': state, 'owners_count': owners_count, 'pts': pts}

# функция собирает все имена брэндов
def get_marks(html):
    soup = BeautifulSoup(html, 'html.parser')
    marks = soup.find_all('a', class_='IndexMarks__item', href=True)
    mark_names = []
    for mark in marks:
        name = mark.find('div', class_='IndexMarks__item-name').text
        url = mark['href'].replace('all', 'used')
        mark_names.append({'name': name, 'url': url})

    return mark_names

# функция получает список страниц брэнда и проходит по ним
def get_mark_data(browser, mark, year_from, page_from=1, already_found=0):
    # enter into specific mark page
    url = mark['url']
    url_gen = url + '?year_from=' + str(year_from)
    mark_name = mark['name']

    word = 'cars/'
    start = url.index(word, 0) + len(word)
    end = url.index('/', start)
    trend = url[start:end]
    parsing_unixtime = int(time.time())

    print(url_gen)
    browser.get(url_gen)
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # get car's count
    pattern = re.compile('\d+')
    count_text = clear_text(
        soup.find('span', class_='ButtonWithLoader__content'))
    count_total = 0
    res = pattern.findall(count_text)
    if len(res) > 0:
        count_total = int(res[0])
    print('count_total=', count_total)

    found_cars = already_found
    page = page_from
    while found_cars < count_total:
        # prepare next page url
        page_url = url_gen + '&page=' + str(page)
        print(page_url)
        page += 1
        # retrieve nex page
        browser.get(page_url)
        # retrieve data of cars on the page
        models, prices, years, mileages, bodyTypes, colors, engines, \
            transmissions, drives, wheels, states, owners, ptss = get_page_data(
                browser)
        # write to appropriate file with name of mark
        write_csv(trend, parsing_unixtime, models, prices, years, mileages, bodyTypes, colors, engines,
                  transmissions, drives, wheels, states, owners, ptss)
        # add found cars
        found_cars += len(models)

        # wait 1 second before retrieve next page
        time.sleep(1)

    print(f'finished download {mark_name} - {found_cars} rows')

# функция делает логин на auto.ru и запускает процесс получения данных
def do_login_and_get_marks():
    browser = webdriver.Firefox(executable_path=PROJECT_PATH+'geckodriver.exe')
    browser.get(BASE_URL)
    html = browser.page_source

    # do confirmation
    if len(browser.find_elements_by_id('confirm-button')) > 0:  # pay attention: find_element*s*
        # pay attention: find_element
        browser.find_element_by_id('confirm-button').click()
        print('login')

    # get marks
    browser.get(BASE_URL)
    # open all models
    # pay attention: find_element*s*
    if len(browser.find_elements_by_class_name('IndexMarks__show-all')) > 0:
        browser.find_element_by_class_name(
            'IndexMarks__show-all').click()  # pay attention: find_element

    html = browser.page_source
    marks = get_marks(html)
    return browser, marks

### Логин в AUTO.RU 

In [ ]:
# browser, marks = do_login_and_get_marks()

### Парсинг с AUTO.RU

In [ ]:
# 
# for mark in marks:
#     if mark['name'].upper() in MARKS_TO_PARSE:
#         print(mark)
#         get_mark_data(browser, mark, FROM_YEAR)